# Notebook Server Setup

In order to configure the notebook server, first the latest version of the AWS CLI v2 needs to be installed.

First check to see if the v2 is already installed. Don't worry if it is not installed, the notebook will install it if it not.

In [ ]:
result = !/usr/local/bin/aws --version
print(result[0])

if not "aws-cli/2" in result[0]:
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -qq awscliv2.zip
    !sudo ./aws/install --update
    !rm -rf aws
    !rm awscliv2.zip

In [1]:
!pip install pyathena

## Validate the AWS CLI v2 is installed.

You should see the following result, or something similar, when the next cell is executed:
```
aws-cli/2.x.x Python/3.x.x Linux/4.14.287-148.504.amzn1.x86_64 exe/x86_64.amzn.2018 prompt/off
```
Validate that the return value starts with aws-cli-/2.x.x

In [ ]:
!/usr/local/bin/aws --version

# Validate notebook server permissions
The notebook server needs access to parameter store values for the SSO config, and an S3 bucket for logging.

In [ ]:
import boto3
session = boto3.Session(profile_name='default')
client = session.client('ssm')

s3_response = client.get_parameter(Name='Jupyter-S3')
sso_portal_url_response = client.get_parameter(Name='Jupyter-SSO-Portal-Url')
sso_instance_response = client.get_parameter(Name='Jupyter-SSO-Directory')
sso_instance_arn_response = client.get_parameter(Name='Jupyter-SSO-Instance-Arn')


s3_bucket = s3_response['Parameter']['Value']
sso_portal_url = sso_portal_url_response['Parameter']['Value']
sso_instance = sso_instance_response['Parameter']['Value']
sso_instance_arn = sso_instance_arn_response['Parameter']['Value']

print(f"Successfully loaded parameters.")
print(f"s3_bucket={s3_bucket}")
print(f"sso_portal_url={sso_portal_url}")
print(f"sso_instance={sso_instance}")
print(f"sso_instance_arn={sso_instance_arn}")



## Validate permissions to IAM Identity Center

The default profile needs access to the IAM identity center to create the profiles on the notebook server for the AWS CLI and SDK. The profiles allow the notebook to configure which profile will be used, allowing the notebooks to be used across accounts.

In the event that a profile is not configured on the notebook server, the notebook server will use the default profile to query  IAM Identity Center all the accounts and permission sets, and construct the profiles. This eliminates the need to call `aws configure sso` for every permutation of account and permission set.

If you get the error **ERROR: No Accounts Associated to upyter-IR-ViewOnly permission set.**, that means there are no AWS accounts associated with the **Jupyter-IR-ViewOnly** permission set. Go into the IAM Identity Center and associate the AWS account to the **Jupyter-IR-ViewOnly** permission set and a user or group.

In [ ]:
sso_admin_client = session.client('sso-admin')
permission_sets = sso_admin_client.list_permission_sets(InstanceArn=sso_instance_arn)

test_account = ""

for permission_set in permission_sets["PermissionSets"]:
    permission_set_detail = sso_admin_client.describe_permission_set(
            InstanceArn=sso_instance_arn,
            PermissionSetArn=permission_set
        )
    accounts = sso_admin_client.list_accounts_for_provisioned_permission_set(
            InstanceArn=sso_instance_arn,
            PermissionSetArn=permission_set
        )
    if permission_set_detail["PermissionSet"]["Name"] == "Jupyter-IR-ViewOnly":
        if len(accounts["AccountIds"]) == 0:
            print(f'ERROR: No Accounts Associated to {permission_set_detail["PermissionSet"]["Name"]} permission set')
        test_account = accounts["AccountIds"][0]
    print(f'Permission Set: {permission_set_detail["PermissionSet"]["Name"]} - Accounts: {accounts["AccountIds"]}')
    

# Test SSO Login

The instance has all the permissions required, now run the login command against the `Jupyter-IR-ViewOnly` role

In [ ]:
import jupyter_aws_sso

import importlib
importlib.reload(jupyter_aws_sso)

jupyter_aws_sso.login("Jupyter-IR-ViewOnly", test_account)

In [ ]:
#session = boto3.Session(profile_name='ManagedPermissionSet-383086473915')
boto3.setup_default_session()
sts = boto3.client('sts')
sts.get_caller_identity()

In [ ]:
! aws sts get-caller-identity


In [ ]:
!/usr/local/bin/aws sso logout

In [ ]:
boto3.setup_default_session()
sts = boto3.client('sts')
try:
    sts.get_caller_identity()
    print("The logout FAILED")
except:
    print("The logout was successful")